In [48]:
import pickle as pkl
import numpy as np
import random

In [49]:
def load_data_training(file_name):

    with open(file_name, 'rb') as f:
        data = pkl.load(f)
    
    training_data = []

    training_cats = data['train']['cat'] / np.max(data['train']['cat'])    
    training_no_cats = data['train']['no_cat']/ np.max(data['train']['no_cat'])

    
    for i in range(len(training_cats)):
        t = list(training_cats[i].flatten())
        training_data.append((t,1))

    for i in range(len(training_no_cats)):
        m = list(training_no_cats[i].flatten())
        training_data.append((m, 0))
    
    random.shuffle(training_data)
    
    return training_data

training = load_data_training('cat_data.pkl')
len(training)

209

In [51]:
def load_data_testing(file_name):

    with open(file_name, 'rb') as f:
        data = pkl.load(f)
    
    testing_data = []

    testing_cats = data['test']['cat'] / np.max(data['test']['cat'])    
    testing_no_cats = data['test']['no_cat']/ np.max(data['test']['no_cat'])
   
    for i in range(len(testing_cats)):
        m = list(testing_cats[i].flatten())
        testing_data.append((m,1))

    for i in range(len(testing_no_cats)):
        m = list(testing_no_cats[i].flatten())
        testing_data.append((m, 0))
    
    random.shuffle(testing_data)
    
    return testing_data

testing = load_data_testing('cat_data.pkl')
len(testing)

In [95]:
import numpy as np
import random

def save_pkl(file_name, data):
            with open(file_name, 'wb') as f:
                pkl.dump(data, f)

class Neuron:

    def __init__(self, dimension=1, weights=None, bias=None, activation=(lambda x: x), predict=(lambda x: x)):
        self._dim = dimension
        self.w = weights or np.array([random.random()*(-1)**random.randint(0, 1) for _ in range(self._dim)])
        self.w = np.array([float(w) for w in self.w])
        self.b = bias if bias is not None else random.random()*(-1)**random.randint(0, 1)
        self.b = float(self.b)
        self._a = activation
        self.predict = predict.__get__(self)

    def __str__(self):
        return "Simple cell neuron\n\
        \tInput dimension: %d\n\
        \tBias: %f\n\
        \tWeights: %s\n\
        \tActivation: %s" % (self._dim, self.b, self.w, self._a.__name__)

    def __call__(self, x):
    
        yhat = self._a(np.dot(self.w, np.array(x)) + self.b)
        return yhat
    
class Trainer:

    def __init__(self, dataset, model, loss):
    
        self.dataset = dataset
        self.model = model
        self.loss = loss

    def validate(self, data):

        results = [self.loss(self.model.predict(x), y) for x, y in data]
        return float(sum(result for result in results))/float(len(results))
    
    def accuracy(self, data):
        return 100*float(sum([1 for x, y in data if self.model.predict(x) == y]))/float(len(data))

    def train(self, lr, ne):

        print("training model on data...")
        accuracy = self.accuracy(self.dataset)
        print("initial accuracy: %.3f" % (accuracy))
        for epoch in range(ne):
            for d in self.dataset:
                x, y = d
                x = np.array(x)
                yhat = self.model(x)
                error = y - yhat
                self.model.w += lr*(y-yhat)*x
                self.model.b += lr*(y-yhat)
            accuracy = self.accuracy(self.dataset)
            print('>epoch=%d, learning_rate=%.3f, accuracy=%.3f' % (epoch+1, lr, accuracy))
        print("training complete")
        print("final accuracy: %.3f" % (self.accuracy(self.dataset)))

# activation functions

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def perceptron(z):
    return -1 if z<=0 else 1

# loss functions

def lrloss(yhat, y):
    return -1.0*(y*np.log(yhat)+(1-y)*np.log(1-yhat))

def ploss(yhat, y):
    return max(0, -yhat*y)

# prediction functions

def ppredict(self, x):
    return self(x)

def lrpredict(self, x):
    return 1 if self(x)>0.5 else 0
    
# extra

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))

In [102]:
neuron = Neuron(dimension = 64*64*3, activation = sigmoid, predict = lrpredict)
trainer = Trainer(training, neuron, lrloss)
updates = trainer.train(0.01, 130)

training model on data...
initial accuracy: 52.632
>epoch=1, learning_rate=0.010, accuracy=59.809
>epoch=2, learning_rate=0.010, accuracy=66.986
>epoch=3, learning_rate=0.010, accuracy=70.335
>epoch=4, learning_rate=0.010, accuracy=61.722
>epoch=5, learning_rate=0.010, accuracy=58.373
>epoch=6, learning_rate=0.010, accuracy=71.292
>epoch=7, learning_rate=0.010, accuracy=61.722
>epoch=8, learning_rate=0.010, accuracy=63.158
>epoch=9, learning_rate=0.010, accuracy=85.167
>epoch=10, learning_rate=0.010, accuracy=67.464
>epoch=11, learning_rate=0.010, accuracy=69.378
>epoch=12, learning_rate=0.010, accuracy=89.474
>epoch=13, learning_rate=0.010, accuracy=80.383
>epoch=14, learning_rate=0.010, accuracy=67.464
>epoch=15, learning_rate=0.010, accuracy=92.344
>epoch=16, learning_rate=0.010, accuracy=69.378
>epoch=17, learning_rate=0.010, accuracy=92.344
>epoch=18, learning_rate=0.010, accuracy=92.344
>epoch=19, learning_rate=0.010, accuracy=86.124
>epoch=20, learning_rate=0.010, accuracy=92.82

NameError: name 'cat_bw' is not defined

In [104]:
cats_bw = list([neuron.b, *neuron.w])
save_pkl('cats_bw.pkl', cats_bw)

with open('cats_bw.pkl', 'rb') as f:
    loadedneuron = pkl.load(f)

len(loadedneuron)

12289

In [105]:
class Tester:

    def __init__(self, dataset, model, loss):
    
        self.dataset = dataset
        self.model = model
        self.loss = loss

    def validate(self, data):

        results = [self.loss(self.model.predict(x), y) for x, y in data]
        return float(sum(result for result in results))/float(len(results))
    
    def accuracy(self, data):
        return 100*float(sum([1 for x, y in data if self.model.predict(x) == y]))/float(len(data))

    def test(self):
        print("testing model on data...")
        accuracy = self.accuracy(self.dataset)
        print("initial accuracy: %.3f" % (accuracy))
        for d in self.dataset:
            x, y = d
            x = np.array(x)
            yhat = self.model(x)
            error = y - yhat
            accuracy = self.accuracy(self.dataset)
        print('accuracy=%.3f' % (accuracy))
        print("testing complete")
        print("final accuracy: %.3f" % (self.accuracy(self.dataset)))
        
# activation function

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# loss function

def lrloss(yhat, y):
    return -1.0*(y*np.log(yhat)+(1-y)*np.log(1-yhat))

# prediction function

def lrpredict(self, x):
    return 1 if self(x)>0.5 else 0

In [106]:
tester = Tester(testing, neuron, lrloss)
tester.test()

testing model on data...
initial accuracy: 60.000
accuracy=60.000
testing complete
final accuracy: 60.000
